<a href="https://colab.research.google.com/github/MoreiraErick/AI/blob/crewAI/sample002_CrewAI_with_GoogleGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Gemini

Este código utiliza a API ChatGoogleGenerativeAI da Google e ferramentas de busca para criar um agente especializado em criptomoedas, especificamente para buscar o preço atual do Bitcoin (BTC) em dólares americanos (USD). O agente utiliza a biblioteca requests para fazer uma requisição à API CoinGecko, que retorna o preço atual do Bitcoin. O fluxo de execução do código inclui a definição do agente, a criação de uma tarefa para buscar o preço do Bitcoin, e a inicialização de uma equipe (Crew) que executa esta tarefa. A saída final é uma string formatada com o preço atual do Bitcoin, ou uma mensagem de erro se a requisição falhar.

This code uses Google's ChatGoogleGenerativeAI API and search tools to create an agent specialized in cryptocurrencies, specifically to fetch the current price of Bitcoin (BTC) in US dollars (USD). The agent uses the requests library to make a request to the CoinGecko API, which returns the current price of Bitcoin. The execution flow of the code includes defining the agent, creating a task to fetch the Bitcoin price, and initializing a crew that executes this task. The final output is a formatted string with the current Bitcoin price, or an error message if the request fails.

In [ ]:
# Instalar pacotes necessários
# Install required packages
!pip install -U duckduckgo-search langchain_google_genai crewai

In [ ]:
# Importar pacotes necessários
# Import necessary packages
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process
from langchain.tools import tool
import requests

# Usado para armazenar sua chave de API com segurança
# Used to securely store your API key
from google.colab import userdata

# Solicitar ao usuário que insira sua chave de API do Google
# Prompt user to enter their Google API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')  # Ou substitua pela sua chave de API do Google / Or replace with your Google API key

# Configurar gemini pro como LLM (Modelo de Linguagem)
# Set gemini pro as LLM (Language Model)
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    verbose=True,
    temperature=0.5,
    google_api_key=GOOGLE_API_KEY
)

# Criar ferramentas
# Create tools
@tool
def fetch_bitcoin_price() -> str:
    """
    A tool to fetch the current Bitcoin (BTC) price in USD.

    This function makes a request to the CoinGecko API to retrieve the current price of Bitcoin in United States Dollars (USD). It returns a formatted string with the current price, making it easy to read and utilize this data elsewhere in the system or in reports.

    On success, the function returns a string in the format: "The current Bitcoin price is USD [price]".
    If the request fails for any reason, such as connection issues or API errors, the function returns an error message indicating the problem encountered.

    Success example: "The current Bitcoin price is USD 50000".
    Error example: "Error fetching the Bitcoin price: [error message]".

    This function uses the 'requests' library for the HTTP request. Ensure this library is installed and available in your Python environment.
    """
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an error for bad responses
        data = response.json()
        bitcoin_price = data['bitcoin']['usd']
        return f"The current Bitcoin price is USD {bitcoin_price}"
    except requests.RequestException as e:
        return f"Error fetching the Bitcoin price: {e}"

# Definir ferramentas
# Define tools
tools = [fetch_bitcoin_price]

# Definir um agente
# Define an agent with the assigned tool
crypto_expert_agent = Agent(
    role='Cryptocurrency Expert',  # Especialista em Criptomoedas
    goal='Fetch the current price of Bitcoin',  # Buscar o preço atual do Bitcoin
    backstory='An experienced analyst in the field of cryptocurrency, deeply knowledgeable about market trends and the technical aspects of blockchain technology. Eager to leverage cutting-edge tools to provide accurate and timely financial insights.',  # Um analista experiente no campo das criptomoedas, profundamente conhecedor das tendências do mercado e dos aspectos técnicos da tecnologia blockchain. Ansioso para utilizar ferramentas de ponta para fornecer insights financeiros precisos e oportunos.
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=tools
)

# Criar uma tarefa para este agente
# Create a task for this agent
fetch_bitcoin_price_task = Task(
    description='Fetch the current price of Bitcoin.',  # Buscar o preço atual do Bitcoin.
    agent=crypto_expert_agent,
    expected_output='A string with the current price of Bitcoin.'  # Uma string com o preço atual do Bitcoin.
)

# Inicializar a equipe e adicionar o agente e a tarefa
# Initialize your crew and add the agent and task to it
crew = Crew(
    agents=[crypto_expert_agent],
    tasks=[fetch_bitcoin_price_task],
    verbose=True,
    process=Process.sequential  # Processo sequencial / Sequential process
)

# Fluxo de Execução
# Execution Flow
print("Crew: Working on Bitcoin Price Task")  # Crew: Trabalhando na Tarefa de Buscar Preço do Bitcoin
price_output = crew.kickoff()  # Iniciar a execução da tarefa / Start task execution
print(price_output)  # Exibir o resultado da tarefa / Display the task result
